# XAI ASSIGNMENT 4
# NAME: TIFFANY DEGBOTSE

In [2]:
!pip install numpy==1.25.2 pandas==2.0.3 scikit-learn==1.2.2 shap==0.45.1
!pip install git+https://github.com/MaximeJumelle/ALEPython.git@dev#egg=alepython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Cloning https://github.com/MaximeJumelle/ALEPython.git (to revision dev) to /tmp/pip-install-h4fgxciq/alepython_b478fb7acb3843239efa21be2fbecd93
  Running command git clone --filter=blob:none --quiet https://github.com/MaximeJumelle/ALEPython.git /tmp/pip-install-h4fgxciq/alepython_b478fb7acb3843239efa21be2fbecd93
  Resolved https://github.com/MaximeJumell

In [3]:
# Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Models
import xgboost
from sklearn.model_selection import train_test_split

# XAI
import shap
from alepython import ale_plot
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance

np.random.seed(1)

# Using IMDB DATASET
### The dataset I chose had a problematic row

In [4]:
import pandas as pd

data = pd.read_csv(
    "src/TMDB_movie_dataset_v11.csv",
    engine="python",        # more flexible parser
    on_bad_lines="skip"     # skip problematic rows
)


FileNotFoundError: [Errno 2] No such file or directory: 'src/TMDB_movie_dataset_v11.csv'

# Inspecting the data

In [ ]:
data.head()

## Dropping rows that have missing values in important columns(eye inspecting of features not use of feature importance)

In [ ]:
# Drop rows with missing values in important cols
data = data.dropna(subset=["budget", "popularity", "runtime", "vote_average", "vote_count"])

## The training was taking long so I decided to reduce my dataset
# Sampled Data

In [ ]:
data_sampled = data.sample(5000, random_state=42)


In [ ]:
data_sampled.head()

## Defining target and features
## I chose just four features to help me interpret better since they are figures and would be easier to work with

In [ ]:
X = data_sampled[["budget", "popularity", "runtime", "vote_count"]]
y = data_sampled["vote_average"]

## Training the model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Train RandomForestRegressor model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


## I included this beacause I noticed that after the PDP plots, the x axis wasn't showing the budget in millions of dollars. I read on it and found out PartialDependenceDisplay.from_estimator() internally uses Matplotlib to draw the figure and it has a default tick formatter.

In [ ]:
data_sampled["budget"].describe()


## Partial Dependence Plot for Budget
### From the plot below, we see that a lower budget of around 0- 10 million gives a low rating arou 1.95 but as the budget increases between 20 million and around 100 million, the curve jumps up with the rating being around 2.25. Beyond around 150 million though, the curve flats out which means increasing budget further doesn't change predictions much.

#### Using PartialDependenceDisplay

In [ ]:
# Choosing the feature of interest
features = ["budget"]

# Use PartialDependenceDisplay to plot PDP
PartialDependenceDisplay.from_estimator(model, X_test, features, kind='average') #kind='both'

## Build my own PDP with numpy
## Based on this plot, budget does not meaningfully affect movie ratings. Predictions hover around 1.7 no matter if the budget is tiny or huge.
## There is a spike at budgets near 0 which might be due to noise



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Choose the feature for which you want to plot partial dependence
feature_index = 0  # 0 = budget, 1 = popularity, 2 = runtime, 3 = vote_count
feature_name = X.columns[feature_index]

# Create a grid of values for the chosen feature
feature_values = np.linspace(
    np.min(X.iloc[:, feature_index]),
    np.max(X.iloc[:, feature_index]),
    num=80
)

# Initialize array to store average predictions
average_predictions = np.zeros_like(feature_values)

# Duplicate the dataset to modify feature values
X_modified = X.copy()

# Loop over feature values
for i, value in enumerate(feature_values):
    # Replace chosen feature with the current grid value
    X_modified.iloc[:, feature_index] = value

    # Predict using the modified dataset
    predictions = model.predict(X_modified)

    # Store the average prediction
    average_predictions[i] = np.mean(predictions)

# Plot the partial dependence for the chosen feature
plt.figure(figsize=(8,5))
plt.plot(feature_values, average_predictions, linewidth=2)
plt.xlabel(feature_name)
plt.ylabel("Average predicted vote_average")
plt.title(f"Partial Dependence for {feature_name}")
plt.grid(True)
plt.show()


# Varying Grid Solution
### The analysis reveals that the model has learned a negative relationship between budget and predicted ratings. In particular, most of the variation occurs within the lower-budget range, where even small increases in budget lead to a noticeable decline in predicted ratings. Beyond this range, the effect diminishes, and additional increases in budget have little to no impact as the curve levels off. Importantly, the consistency of this downward trend across multiple grid resolutions suggests that the relationship is stable and not an artifact of the sampling method. I found this suspicious so I decided to plot a correlation matrix

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import numpy as np

# Pick the feature index (0 = budget, 1 = popularity, 2 = runtime, 3 = vote_count)
feature_index = 0
feature_name = X.columns[feature_index]

# Define different grid lengths (how many points we evaluate PDP on)
grid_lengths = np.linspace(20, 120, 40).astype(int)

fig, ax = plt.subplots(figsize=(8, 6))

# Define a base color and alpha (transparency) values
base_color = (0.2, 0.4, 0.6)  # Blue-ish color
min_alpha = 0.2
max_alpha = 1.0

# Normalize grid_lengths between 0 and 1 for transparency scaling
normalized_g = (grid_lengths - grid_lengths.min()) / (grid_lengths.max() - grid_lengths.min())

for i, g in enumerate(grid_lengths):
    # Create feature grid with g points
    feature_values = np.linspace(
        np.min(X.iloc[:, feature_index]),
        np.max(X.iloc[:, feature_index]),
        num=g
    )

    # Initialize array to store average predictions
    average_predictions = np.zeros_like(feature_values, dtype=float)

    # Duplicate the dataset to modify feature values
    X_modified = X.copy()

    # Loop over feature values
    for j, value in enumerate(feature_values):
        X_modified.iloc[:, feature_index] = value
        predictions = model.predict(X_modified)
        average_predictions[j] = np.mean(predictions)

    # Calculate color with alpha based on grid length
    alpha = min_alpha + (max_alpha - min_alpha) * normalized_g[i]
    color = to_rgba(base_color, alpha)

    # Plot the partial dependence curve
    ax.plot(feature_values, average_predictions, color=color)

ax.set_xlabel(feature_name)
ax.set_ylabel("Average predicted vote_average")
ax.set_title(f"Partial Dependence for {feature_name}\n(Varying Grid Resolution)")
plt.show()


## Correlation Matrix
The correlation matrix shows that budget and vote_count have the strongest positive relationship (0.60), indicating that higher-budget films typically receive more votes. Other feature pairs, such as budget with runtime (0.25) and budget with popularity (0.22), show only weak correlations. Overall, aside from the overlap between budget and vote_count, the features are largely independent, suggesting minimal multicollinearity in the dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Compute correlation matrix
corr_matrix = X.corr()

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Matrix")
plt.show()


## ICE PLOT

In [ ]:
# Choosing the feature of interest
features = ["budget"]

# Use PartialDependenceDisplay to plot PDP
PartialDependenceDisplay.from_estimator(model, X_test, features, kind='individual')

## ALE Plots 📉

Accumulated Local Effects (ALE) Plots [Paper, 2020](https://www.scholars.northwestern.edu/en/publications/visualizing-the-effects-of-predictor-variables-in-black-box-super)

**How to create an ALE plot:**
1. Bin the Feature: Divide the feature of interest into several intervals (bins). These bins help in managing the data and computing local effects within smaller, more manageable segments.
2. Compute Local Effects: For each bin, calculate the local effect of the feature on the prediction. This involves: Calculating the change in prediction when moving from the lower to the upper edge of the bin
Averaging this change over all instances that fall into that bin
3. Accumulate Effects: Starting from the first bin, accumulate the local effects across all bins. Sum up the average effects sequentially to show how the feature influences the prediction as its value changes
4. Centering: To make the plot more interpretable, center the accumulated effects around zero. Subtract the mean of the accumulated effects, which forces the interpretation to focus on deviations from the average prediction

There are a few python implementations of ALE plots, here we show the [ALEPython implementation](https://github.com/blent-ai/ALEPython).

The implementation is more complex and less intuitive than PDPs, with many hyperparameters, including:
* **bins** - This parameter defines the number of bins to divide the range of the specified feature into. A larger number of bins can provide more granularity in the ALE plot but might also increase computation time.
* **monte_carlo** - This parameter is a boolean flag indicating whether to use Monte Carlo sampling to estimate the ALE. Monte Carlo sampling can be beneficial when the number of samples in the dataset is large, as it reduces computational burden.
* **monte_carlo_rep** - This parameter specifies the number of Monte Carlo replicates to use for estimating the ALE. More replicates can lead to a more accurate estimation but may also increase computation time.
* **monte_carlo_ratio** - This parameter determines the proportion of the dataset to use for Monte Carlo sampling. It's a value between 0 and 1, where 1 means using the entire dataset. Using a smaller ratio can speed up computation but may introduce some sampling error.


This can take a couple of minutes to run:

In [ ]:
# Use default parameters for 1D Main Effect ALE Plot
ale_plot(model, X_train, 'budget', monte_carlo=True)

In [ ]:
# Change hyperparameters for 1D Main Effect ALE Plot
ale_plot(
    model,
    X_train,
    "budget",
    bins=5,
    monte_carlo=True,
    monte_carlo_rep=30,
    monte_carlo_ratio=0.5,
)

#### How to Interpret 1D Main Effect ALE Plot



* X-axis represents feature values
* Y-axis shows average effect on predictions
* Each curve represents a feature's ALE. - Flat curves imply little impact; steep curves, significant impact
* Upward curves: increasing feature value increases predictions; downward, the opposite
* Steeper curves signify larger effects

We can compare ALE plots to gauge relative feature importance.
Features with steeper curves have larger impacts.

In [ ]:
# 2D Second-Order ALE Plot
ale_plot(model, X_train, X_train.columns[:2], bins=10)

#### How to Interpret - 2D Second-Order ALE Plot

* Both axes represent the values of the two features being analyzed.
* Each axis corresponds to one of the features.
* The plot displays a surface where the height represents the average effect on predictions. Higher points indicate regions where the model tends to make higher predictions, and vice versa.
* Patterns in the surface reveal how the joint behavior of the two features affects the model's predictions. Peaks or valleys suggest regions where the joint effect is particularly strong.
* The direction of the slope indicates whether increasing one feature while holding the other constant tends to increase or decrease predictions. Steeper slopes represent larger effects, while flatter regions indicate smaller effects.


We can compare the 2D Second-Order ALE Plot with individual ALE plots for each feature to understand how joint effects differ from marginal effects.